In [1]:
import requests
import pandas as pd

In [2]:
df = pd.read_csv("../data/processed/02_housing_data_enriched_fred.csv")
df['Date'] = pd.to_datetime(df['Date'])

In [3]:
url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$limit=5000"
response = requests.get(url)
data = response.json()
fema_df = pd.json_normalize(data["DisasterDeclarationsSummaries"])
fema_df.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5529-OR,5529,OR,FM,2024-08-09T00:00:00.000Z,2024,Fire,LEE FALLS FIRE,False,False,...,99067,Washington (County),24122,None,2024081001,10,R,2024-08-27T18:22:14.800Z,ae87cf3c6ed795015b714af7166c7c295b2b67c7,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
1,FM-5528-OR,5528,OR,FM,2024-08-06T00:00:00.000Z,2024,Fire,ELK LANE FIRE,False,False,...,99031,Jefferson (County),24116,None,2024080701,10,R,2024-08-27T18:22:14.800Z,432cf0995c47e3895cea696ede5621b810460501,59983f89-30bf-4888-b21b-62e8d57d9aac
2,FM-5527-OR,5527,OR,FM,2024-08-02T00:00:00.000Z,2024,Fire,MILE MARKER 132 FIRE,False,False,...,99017,Deschutes (County),24111,None,2024080301,10,R,2024-08-27T18:22:14.800Z,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
3,DR-4312-CA,4312,CA,DR,2017-05-02T00:00:00.000Z,2017,Severe Storm,FLOODING,False,False,...,60347,Resighini Rancheria (Indian Reservation),17035,None,2017041001,9,None,2025-03-26T20:21:32.579Z,432a3a64bdbb291ae26cf5a27a33deeabb380481,98a7c5bb-2346-45aa-a1ca-0399440d4f0b
4,DR-4251-AL,4251,AL,DR,2016-01-21T00:00:00.000Z,2016,Severe Storm,"SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS,...",False,False,...,99001,Autauga (County),16003,None,2015122301,4,None,2025-03-27T12:21:46.559Z,dcd4ce6b37ee49875b3f1e32e9a8a16cd6a803d3,5229bbae-eee6-42b8-b277-edbafa8d6cb2


In [4]:
fema_df = fema_df[[
    'state', 'incidentType', 'declarationDate', 'declarationType'
]]
fema_df['declarationDate'] = pd.to_datetime(fema_df['declarationDate'])
fema_df['declarationDate'] = fema_df['declarationDate'].dt.tz_localize(None)
fema_df = fema_df[fema_df['declarationType'] == 'DR']  # Only major disasters
fema_df.head()

,state,incidentType,declarationDate,declarationType
3,CA,Severe Storm,2017-05-02,DR
4,AL,Severe Storm,2016-01-21,DR
5,AL,Severe Storm,2016-01-21,DR
6,AL,Severe Storm,2016-01-21,DR
11,AL,Severe Storm,2016-01-21,DR


In [5]:
state_month_combos = df[['StateName', 'Date']].drop_duplicates()
disaster_features = []

for _, row in state_month_combos.iterrows():
    state = row['StateName']
    current_date = row['Date']
    start_window = current_date - pd.DateOffset(months=12)

    recent_disasters = fema_df[
        (fema_df['state'] == state) &
        (fema_df['declarationDate'] >= start_window) &
        (fema_df['declarationDate'] <= current_date)
    ]

    disaster_features.append({
        'StateName': state,
        'Date': current_date,
        'Num_Major_Disasters_Last_12mo': recent_disasters.shape[0],
        'Major_Disaster_Declared': int(not recent_disasters.empty),
        'Most_Recent_Disaster_Type': recent_disasters.sort_values(
            'declarationDate', ascending=False
        ).incidentType.iloc[0] if not recent_disasters.empty else None
    })

disaster_df = pd.DataFrame(disaster_features)

In [6]:
disaster_df.head()

,StateName,Date,Num_Major_Disasters_Last_12mo,Major_Disaster_Declared,Most_Recent_Disaster_Type
0,US,2018-01-31,0,0,None
1,NY,2018-01-31,0,0,None
2,CA,2018-01-31,1,1,Severe Storm
3,IL,2018-01-31,0,0,None
4,TX,2018-01-31,0,0,None


In [7]:
df_merged = df.merge(disaster_df, on=['StateName', 'Date'], how='left')

df_merged.head()

,RegionID,RegionName,RegionType,StateName,SizeRank,Date,ZHVI_AllHomes,ZHVI_SingleFamily,ZHVI_Condo,SalesCount_SFR,Inventory_SFR,Year,Month,Unemployment_Rate,Median_Income,Population,Num_Major_Disasters_Last_12mo,Major_Disaster_Declared,Most_Recent_Disaster_Type
0,102001,United States,country,US,0,2018-01-31,222399.430599,221704.438544,232520.632007,248552.0,1206651.0,2018,1,4.0,75790.0,327969.000,0,0,None
1,394913,"New York, NY",msa,NY,1,2018-01-31,468289.939148,458621.808084,496261.053175,10754.0,49048.0,2018,1,4.5,80700.0,19544.098,0,0,None
2,753899,"Los Angeles, CA",msa,CA,2,2018-01-31,613037.032936,637911.684582,469858.117823,5302.0,16116.0,2018,1,4.4,84560.0,39437.463,1,1,Severe Storm
3,394463,"Chicago, IL",msa,IL,3,2018-01-31,227029.700914,231794.550568,190146.965317,5550.0,27301.0,2018,1,4.5,84140.0,12724.685,0,0,None
4,394514,"Dallas, TX",msa,TX,4,2018-01-31,238849.482465,240010.957031,163186.511511,5077.0,22001.0,2018,1,4.1,71720.0,28624.564,0,0,None


In [8]:
df_merged.to_csv("../data/processed/03_housing_data_enriched_fred_fema.csv", index=False)

print("File successfully saved to 'data/processed/03_housing_data_enriched_fred_fema.csv'")

File successfully saved to 'data/processed/03_housing_data_enriched_fred_fema.csv'
